In [1]:
from tensorflow.examples.tutorials.mnist import input_data

/Users/MG/.pyenv/versions/3.6.4rc1/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/Users/MG/.pyenv/versions/3.6.4rc1/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/MG/.pyenv/versions/3.6.4rc1/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


In [2]:
mnist=input_data.read_data_sets('./MNIST_data', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
mnist.train.images.shape

(55000, 784)

In [4]:
mnist.test.images.shape

(10000, 784)

In [6]:
X_train, y_train=mnist.train.next_batch(2000)

In [7]:
X_train.shape

(2000, 784)

In [20]:
X_test, y_test=mnist.test.next_batch(500)

In [21]:
X_test.shape

(500, 784)

## 打印

In [8]:
import matplotlib.pyplot as plt

In [ ]:
plt.rc('image',cmap='binary')
fot i in range(10):
    plt.subplot(2,5,i+1)
    

## 全连接网络

In [10]:
from keras.models import Sequential
from keras.layers import Dense

In [14]:
def model1():
    model=Sequential()
    model.add(Dense(784,activation='relu'))
    model.add(Dense(100,activation='relu'))
    model.add(Dense(10,activation='softmax'))
    return model

In [15]:
m=model1()

In [17]:
m.summary

<bound method Network.summary of <keras.engine.sequential.Sequential object at 0x140951908>>

In [18]:
m.compile(optimizer='adam',loss='categorical_crossentropy')

Instructions for updating:
Colocations handled automatically by placer.


In [19]:
history=m.fit(X_train, y_train, epochs=10,batch_size=32)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
2000/2000 [==============================] - 3s 2ms/step - loss: 0.7922
Epoch 2/10
2000/2000 [==============================] - 1s 701us/step - loss: 0.2693
Epoch 3/10
2000/2000 [==============================] - 1s 631us/step - loss: 0.1728
Epoch 4/10
2000/2000 [==============================] - 1s 647us/step - loss: 0.0849
Epoch 5/10
2000/2000 [==============================] - 1s 615us/step - loss: 0.0450
Epoch 6/10
2000/2000 [==============================] - 1s 591us/step - loss: 0.0225
Epoch 7/10
2000/2000 [==============================] - 1s 604us/step - loss: 0.0105
Epoch 8/10
2000/2000 [==============================] - 1s 502us/step - loss: 0.0056
Epoch 9/10
2000/2000 [==============================] - 1s 569us/step - loss: 0.0038
Epoch 10/10
2000/2000 [==============================] - 1s 646us/step - loss: 0.0030


In [22]:
pred=m.predict(X_test)

In [23]:
from sklearn.metrics import accuracy_score

In [24]:
accuracy_score(pred.argmax(1), y_test.argmax(1))

0.93

## LeNet

In [25]:
import torch

In [26]:
import torch.nn as nn

In [27]:
import torch.nn.functional as F

In [28]:
import torch.optim as optim

In [29]:
import torchvision

In [33]:
import torchvision.transforms as transforms

In [31]:
import torchvision.datasets as datasets

In [32]:
import os
import random
import numpy as np

In [46]:
data_train=transforms.Compose([
    transforms.Resize(32),
    transforms.ToTensor(),
    #transforms.Normalize((0.13070016,),(0.30815944,)) #<-参数mean和std来自训练集，但是transform本身会在训练集和评测的时候使用
])

#### 图片的Normalize

目的：将图片进行归一化的缩放（x-mean）/std

思考：图片归一化后，真的不存小于0或者大于1 的outlier了吗？

不是，有小于0，大于1

思考：归一化哪部分数据？A训练集 B评测集 C训练集+评测集？
A

不是C，永远不可能全知全能拿到所有数据


评测集不归一化，我们用归一化训练出的模型能用吗？

量纲拉到训练集的量纲


比赛时，test是不知道的，归一化中的outlier



这么说评测集也是归一化的，只是归一化的时候，不是用的所有数据的men和std？

永远不可能拿到所有数据集

标准化、归一化、正则化


问题应该表述为，用训练集的mean, std 做归一化。而不是用评测+训练集的mean, std 做归一化。

In [44]:
np.mean(mnist.train.images)

0.13070016

In [50]:
np.std(mnist.train.images)

0.30815944

In [52]:
data_trans=transforms.Compose([
    transforms.Resize(32),
    transforms.ToTensor(),
    transforms.Normalize((0.13070016,),(0.30815944,)) #<-参数mean和std来自训练集，但是transform本身会在训练集和评测的时候使用
])

In [54]:
train_data=datasets.MNIST('data',train=True,download=True,transform=data_trans)

100.1%

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz


113.5%

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz


100.4%

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz


180.4%

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [71]:
test_data=datasets.MNIST('data',train=False,download=True,transform=data_trans)

In [66]:
n_train=int(len(train_data)*0.9)
n_valid=len(train_data) - n_train

In [67]:
train_data, valid_data=torch.utils.data.random_split(train_data, [n_train, n_valid])

In [72]:
print(len(train_data), len(valid_data), len(test_data))

43740 4860 10000


In [ ]:
batch_size=64

完成了数据集的制作

In [ ]:
train_iterator=torch.utils.data.DataLoader(train_data,shuffle=True,batch_size=batch_size)
valid_iterator=torch.utils.data.DataLoader(valid_data,batch_size=batch_size)
test_iterator=torch.utils.data.DataLoader(test_data,batch_size=batch_size)

In [ ]:
class LeNet(nn.Model):
    def __int__(self):
        super(LeNet, self).__int__()
        
        

## 载入函数并训练

In [ ]:
optimizer=optim.Adam(model.parameters())

In [ ]:
loss=nn.CrossEntropyLoss()

In [ ]:
### 如何评测结果集--计算精度

In [ ]:
def accu(fx, y):
    pred = fx.max(1, keepdim=True)[1]
    correct = pred.eq(y.view_as(preds)).sum() # 得到该batch的精度
    acc=correct.float()/preds.shape[0]
    return acc
    

In [ ]:
def train(model, device, iterator, optimizer, loss):
    epoch_loss = 0 #积累变量
    epoch_acc = 0 
    model.train() # PHASE=Train
    
    for (x, y) in iterator: # 哪一个minibatch
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        fx = modeel(x)
        loss = loss(fx, y) #计算Loss,train_loss
        acc = accu(fx, y) #计算精确度,train_accu
        loss.backword() #进行BP
        optimizer.step() #统一更新模型
        epoch_loss+=loss.item()
        epoch_acc+=acc.item()
        
    return epoch_loss/len(iterator), epoch_acc/len(iterator)

In [ ]:
def evaluate(model, device, iterator, loss):
    

## 开始训练

In [74]:
epochs = 10
model_dir='models'

In [76]:
best_valid_loss=float('inf')

In [79]:
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)

In [ ]:
for epoch in range(epochs):
    train_loss, train_acc = train(model, device=)